In [ ]:
# https://www.reddit.com/r/computervision/comments/kfhc3u/how_does_one_fine_tune_cnn_hyperparameter_when/
# Guarda tipo di genetic algorithms (YOLO)
# Tuning hyperparameters in the context of large datasets can be a problem. I should investigate further.